# Fine-tuning Flan-T5-base for Legal Document Summarization - google colab version

This notebook is a modified version of the original notebook by Gourab S. (@heygourab)

Author: Gourab S.(@heygourab),

<a href="https://github.com/heygourab"><img src="https://github.com/fluidicon.png" width="20" height="20" alt="github" /> @heygourab</a>

This notebook demonstrates fine-tuning the Flan-T5-base model on the BillSum dataset using LoRA (Low-Rank Adaptation). We'll use the Hugging Face ecosystem (`transformers`, `datasets`, `peft`) for efficient fine-tuning.

## Setup Overview

- Base Model: google/flan-t5-base
- Dataset: BillSum (~2000 samples)

## Prerequisites

This notebook assumes a Colab environment with a GPU available. If you're running this locally, make sure to install the required packages and set up your GPU environment accordingly.

<a href="https://colab.research.google.com/github/heygourab/pdf_summarization_model_fine_tuning/blob/main/notebooks/billsum_lora_finetune_colab.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>


## 1. Environment Setup

First, let's install the required dependencies and set up GPU monitoring.


In [1]:
# Install required packages
%pip install -q transformers datasets accelerate evaluate peft bitsandbytes bert_score rouge_score nltk wandb omegaconf torch 
%pip install -U bitsandbytes -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries


In [2]:
import os
import json
import torch
import nltk
import evaluate
import numpy as np
import sys
import logging
from datetime import datetime
from datasets import load_dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
import wandb
import psutil

## 3. Logger setup


In [4]:
def setup_logger(name="train_logger", level=logging.INFO, log_file=None):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.propagate = False  # Avoid duplicate logs

    # Clear existing handlers
    if logger.hasHandlers():
        logger.handlers.clear()

    # Formatter for log messages
    formatter = logging.Formatter(
        fmt='%(asctime)s — %(name)s — %(levelname)s — %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

    # Console handler setup
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    # File handler setup
    if log_file is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        log_dir = os.path.join(os.getcwd(), 'logs')  # Safe fallback to current dir
        os.makedirs(log_dir, exist_ok=True)
        log_file = os.path.join(log_dir, f'training_{timestamp}.log')
    else:
        log_dir = os.path.dirname(log_file)
        if log_dir:
            os.makedirs(log_dir, exist_ok=True)

    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    # Log header
    logger.info(f"Logger initialized: {name}")
    logger.info(f"Log file created at: {os.path.abspath(log_file)}")
    logger.info(f"Python version: {sys.version}")

    return logger

# Use it
logger = setup_logger("train_logger", logging.INFO)

2025-05-19 13:52:07 — train_logger — INFO — Logger initialized: train_logger


2025-05-19 13:52:07 — train_logger — INFO — Log file created at: /Users/gourabsarkar/Developer/college_project/pdf_summarization_model_fine_tuning/notebooks/logs/training_20250519_135207.log
2025-05-19 13:52:07 — train_logger — INFO — Python version: 3.10.17 (main, Apr  8 2025, 12:10:59) [Clang 16.0.0 (clang-1600.0.26.6)]


## 4. Loading the required NLTK libraries


In [5]:
# Download required NLTK data
for resource in ['punkt', 'punkt_tab']:
    try:
        nltk.download(resource, quiet=True)
        logger.info(f"Successfully downloaded NLTK resource: {resource}")
    except Exception as e:
        logger.error(f"Error downloading {resource}: {e}")

2025-05-19 13:52:31 — train_logger — INFO — Successfully downloaded NLTK resource: punkt
2025-05-19 13:52:33 — train_logger — INFO — Successfully downloaded NLTK resource: punkt_tab


## 5. Memory Usage Monitoring

The `print_memory_usage()` function monitors system resource utilization during model training:

- Tracks RAM usage by getting the Resident Set Size (RSS) of current process in GB
- For GPU-enabled systems:
  - Reports allocated GPU memory
  - Shows total available GPU memory
  - Calculates percentage of GPU memory utilization
  - Resets peak memory tracking statistics

This helps identify potential memory bottlenecks and optimize resource usage during training.


In [6]:
def print_memory_usage():
    process = psutil.Process(os.getpid()) # Get the current process

    ram_gb = process.memory_info().rss / 1e9 # Convert bytes to GB
    total_gb = psutil.virtual_memory().total / 1e9 # Total system RAM in GB

    logger.info(f"RAM usage: {ram_gb:.2f} GB") # Current process RAM usage
    logger.info(f"Total system RAM: {total_gb:.2f} GB") # Total system RAM

    if torch.cuda.is_available():
        torch.cuda.synchronize()
        gpu_mem = torch.cuda.memory_allocated() / 1e9
        gpu_total = torch.cuda.get_device_properties(0).total_memory / 1e9
        peak_gpu_mem = torch.cuda.max_memory_allocated() / 1e9

        logger.info(f"GPU memory usage: {gpu_mem:.2f}/{gpu_total:.2f} GB ({gpu_mem/gpu_total*100:.1f}%)")
        logger.info(f"Peak GPU memory: {peak_gpu_mem:.2f} GB")

        torch.cuda.reset_peak_memory_stats()

print_memory_usage()

2025-05-19 13:52:43 — train_logger — INFO — RAM usage: 0.04 GB
2025-05-19 13:52:43 — train_logger — INFO — Total system RAM: 8.59 GB


## 6. Configuration Parameters

all hyperparameters and configuration settings for the model, dataset, LoRA, and training.


In [ ]:
from bitsandbytes import BitsAndBytesConfig
from transformers import TaskType
from datetime import datetime

CONFIG = {
    # Model & Quantization
    "model_name": "google/flan-t5-base",
    "model_type": "encoder-decoder",
    "quantization_config": BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    ),

    # Dataset
    "dataset_name": "billsum",
    "text_col": "text",
    "summary_col": "summary",
    "max_input_tokens": 512,
    "max_target_tokens": 256,
    "sample_size": 2000,
    "filter_by_length": True,
    "split_train_frac": 0.8,

    # Prompt
    "prompt_prefix": "Summarize: ",

    # LoRA Adapter
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_target_modules": ["q", "v", "k"],
    "lora_dropout": 0.05,
    "lora_bias": "none",
    "lora_task_type": TaskType.SEQ_2_SEQ_LM,

    # Training (Turbo Mode)
    "do_train": True,
    "do_eval": True,
    "num_train_epochs": 3,                 # Reduced for speed
    "per_device_train_batch_size": 16,     # Larger batch
    "per_device_eval_batch_size": 16,
    "gradient_accumulation_steps": 2,      # Fewer accum steps
    "learning_rate": 2e-4,                 # Higher LR for faster convergence
    "weight_decay": 0.05,
    "warmup_steps": 200,
    "fp16": False,
    "bf16": True,                          # If Colab Pro/A100 supports BF16
    "torch_compile": True,                 # PyTorch 2.0+ compilation
    "gradient_checkpointing": True,
    "optim": "adamw_torch",                # Stable optimizer

    # Logging & Checkpointing
    "logging_steps": 100,                  # Log less often
    "evaluation_strategy": "steps",
    "eval_steps": 500,                     # Evaluate less often
    "save_strategy": "steps",
    "save_steps": 1000,                    # Save less often
    "save_total_limit": 3,
    "load_best_model_at_end": True,
    "metric_for_best_model": "rougeL",
    "greater_is_better": True,
    "report_to": "wandb",
    "overwrite_output_dir": True,

    # Generation (Inference)
    "gen_num_beams": 5,
    "gen_length_penalty": 0.8,
    "gen_early_stopping": True,

    # Random seed
    "seed": 42,

    # Google Drive (Colab)
    "mount_drive": True,
    "drive_path": "MyDrive/ML_models/pdf_summarization",
    "training_report_filename": "training_report.json",
    "lora_adapter_name": "lora_billsum_legal",
}

# Timestamped output directories
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
CONFIG["output_dir"] = f"{CONFIG['lora_adapter_name']}_{timestamp}"
if CONFIG["mount_drive"]:
    CONFIG["gdrive_output_dir"] = (
        f"/content/drive/{CONFIG['drive_path']}/{CONFIG['output_dir']}"
    )

# Sanity print
logger.info("Turbo CONFIG:")
for k, v in CONFIG.items():
    logger.info(f"  {k}: {v}")


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/gourabsarkar/Developer/college_project/pdf_summarization_model_fine_tuning/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


ImportError: cannot import name 'BitsAndBytesConfig' from 'bitsandbytes' (/Users/gourabsarkar/Developer/college_project/pdf_summarization_model_fine_tuning/.venv/lib/python3.10/site-packages/bitsandbytes/__init__.py)

## 6. Login to Hugging Face Hub and Weights & Biases

You'll need to log in to Hugging Face to download models/datasets and to Weights & Biases for experiment tracking.
You can get your Hugging Face token from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
and your W&B API key from [https://wandb.ai/authorize](https://wandb.ai/authorize).


In [ ]:
from huggingface_hub import HfFolder, notebook_login

try:
    if HfFolder.get_token() is None:
        logger.info("Hugging Face token not found. Please log in.")
        notebook_login()
    else:
        logger.info("Already logged in to Hugging Face Hub.")
except Exception as e:
    logger.error(f"An error occurred during Hugging Face login check: {e}")
    logger.info("Attempting login...")
    notebook_login()

# Login to Weights & Biases
try:
    wandb.login()
    wandb.init(project="flan-t5-billsum-lora", name=CONFIG["output_dir"], config=CONFIG)
    logger.info("Successfully logged in to W&B and initialized experiment.")
except Exception as e:
    logger.error(f"Could not login to W&B: {e}. Ensure you have run `wandb login` or set WANDB_API_KEY.")
    CONFIG["report_to"] = "tensorboard" # Fallback to tensorboard
    logger.info("Falling back to TensorBoard for logging.")
    # No explicit init for tensorboard here, Trainer handles it via TrainingArguments
# Note: Use environment variables or notebook secrets to store your tokens securely

## 7. Mount Google Drive (Optional)

If you want to save your model checkpoints and outputs to Google Drive, mount it here.


In [ ]:
def is_colab():
    """Check if the current environment is Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        return False
        
if is_colab() and CONFIG["mount_drive"]:
    from google.colab import drive
    try:
        drive.mount('/content/drive')
        logger.info("Google Drive mounted successfully.")
        # Create the output directory on Drive if it doesn't exist
        if not os.path.exists(CONFIG["gdrive_output_dir"]):
            os.makedirs(CONFIG["gdrive_output_dir"], exist_ok=True)
            logger.info(f"Created Google Drive output directory: {CONFIG['gdrive_output_dir']}")
    except Exception as e:
        logger.error(f"Failed to mount Google Drive: {e}")
        logger.info("Proceeding without Google Drive. Outputs will be saved to Colab ephemeral storage.")
        CONFIG["mount_drive"] = False # Disable drive features if mount fails
else:
    logger.info("Not running in Colab or Google Drive is disabled in the configuration.")
    CONFIG["mount_drive"] = False

## 8. Load Model, Tokenizer and Configure LoRA

Loads the Flan-T5-base model and tokenizer from Hugging Face, configures the model for LoRA training and returns the model and tokenizer objects.


In [ ]:
# Install latest bitsandbytes and required dependencies
%pip install -U bitsandbytes --no-cache-dir -q
%pip install accelerate --upgrade -q
%pip install transformers --upgrade -q

# Import required libraries
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
import bitsandbytes as bnb

# Function to verify CUDA and bitsandbytes setup
def verify_installation():
    logger.info(f"PyTorch version: {torch.__version__}")
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    logger.info(f"bitsandbytes version: {bnb.__version__}")
    
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available. 4-bit quantization requires a CUDA-enabled GPU.")
    
    # Test BitsAndBytes CUDA kernels
    try:
        _ = bnb.matmul(torch.zeros(2, 2).cuda(), torch.zeros(2, 2).cuda())
        logger.info("BitsAndBytes CUDA kernels working correctly")
    except Exception as e:
        logger.error(f"BitsAndBytes CUDA test failed: {e}")
        raise

# Verify installation
verify_installation()

# Load tokenizer
logger.info(f"Loading tokenizer for model: {CONFIG['model_name']}")
try:
    tokenizer = AutoTokenizer.from_pretrained(
        CONFIG["model_name"],
        use_fast=True,
        padding_side="right",
        model_max_length=CONFIG["max_input_tokens"]
    )
    logger.info("Tokenizer loaded successfully")
except Exception as e:
    logger.error(f"Failed to load tokenizer: {e}")
    raise


# Configure quantization
logger.info("Configuring 4-bit quantization...")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Changed from bfloat16 for better compatibility
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load model with proper error handling
logger.info(f"Loading base model: {CONFIG['model_name']} with 4-bit quantization...")
try:
    model = AutoModelForSeq2SeqLM.from_pretrained(
        CONFIG["model_name"],
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.float16,  # Match compute dtype
        low_cpu_mem_usage=True
    )
    logger.info("Model loaded successfully")

except Exception as e:
    logger.error(f"Failed to load model: {e}")
    logger.info("Attempting to load without quantization as fallback...")
    try:
        model = AutoModelForSeq2SeqLM.from_pretrained(
            CONFIG["model_name"],
            device_map="auto",
            torch_dtype=torch.float16
        )
        logger.warning("Model loaded without quantization")
    except Exception as e:
        logger.error(f"Failed to load model even without quantization: {e}")
        raise

# Prepare model for k-bit training
logger.info("Preparing model for k-bit training...")
try:
    model = prepare_model_for_kbit_training(
        model,
        use_gradient_checkpointing=CONFIG["gradient_checkpointing"]
    )
    logger.info("Model prepared for k-bit training")
except Exception as e:
    logger.error(f"Failed to prepare model for kbit training: {e}")
    raise

# LoRA configuration and application
lora_config = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    target_modules=CONFIG["lora_target_modules"],
    lora_dropout=CONFIG["lora_dropout"],
    bias=CONFIG["lora_bias"],
    task_type=CONFIG["lora_task_type"],
)

logger.info('Applying LoRA to the model')
try:
    model = get_peft_model(model, lora_config)
    logger.info("LoRA configured and applied to the model")
except Exception as e:
    logger.error(f"Failed to apply LoRA to the model: {e}")
    raise

# Print model statistics
model.print_trainable_parameters()
print_memory_usage()

## 9. Load and Preprocess Dataset

Load the BillSum dataset, preprocess it for Flan-T5, and split into training and evaluation sets.


In [ ]:
import re
def clean_text(text):
    text = text.strip()
    text = " ".join(text.split())
    # Only strip metadata if required
    text = re.sub(r'\s*\([^\)]{0,40}\)\s*', ' ', text)  # remove very short inlines
    text = re.sub(r'\s*\[[^\]]{0,40}\]\s*', ' ', text)
    return text

def preprocess_function(examples):
    try:
        input_texts = examples.get(CONFIG["text_col"], [])
        summary_texts = examples.get(CONFIG["summary_col"], [])

        if not input_texts:
            raise ValueError("Empty input text")

        cleaned_inputs = [clean_text(doc) for doc in input_texts]

        prompts = [f"{CONFIG["prompt_prefix"]} {doc}" for doc in cleaned_inputs]

        print(f"Raw: {input_texts[0][:150]}...")
        print(f"Cleaned: {cleaned_inputs[0][:150]}...")

        model_inputs = tokenizer(
            prompts,
            max_length=CONFIG["max_input_tokens"] - 32,
            padding="max_length",
            truncation=True,
        )

        # Use plain summaries — no extra formatting
        summaries = [s if s else "No summary provided." for s in summary_texts]

        labels = tokenizer(
            summaries,
            max_length=CONFIG["max_target_tokens"],
            truncation=True,
            padding="max_length",
        )

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    except Exception as e:
        print(f"[ERROR] Preprocessing failed: {e}")
        raise


In [ ]:
logger.warning("Attempting to update some libraries.")
%pip install datasets --upgrade -q
%pip install fsspec --upgrade -q
%pip install pyarrow --upgrade -q
logger.warning("Library update attempts finished. If issues persist, ensure runtime was restarted after updates.")


from datasets import load_dataset
import pandas as pd

logger.info(f"Starting dataset loading and processing for: {CONFIG['dataset_name']}")

dataset = None
load_dataset_kwargs = {
    "path": CONFIG["dataset_name"],
    "split": f"train[:{CONFIG['sample_size']}]",
}

try:
    logger.info(f"Loading dataset with streaming=False and split sample size: {CONFIG['sample_size']}")
    dataset = load_dataset(
        **load_dataset_kwargs,
    )
    logger.info("Dataset loaded successfully with streaming=False")

except Exception as e:
    logger.warning(f"Attempt 1 failed: {str(e)}")
    logger.info("Attempting alternative loading method using pandas...")
    try:
        # Define splits and paths
        splits = {
            'train': 'data/train-00000-of-00001.parquet',
            'test': 'data/test-00000-of-00001.parquet',
            'ca_test': 'data/ca_test-00000-of-00001.parquet'
        }
        
        # Load training data using pandas
        df = pd.read_parquet(f"hf://datasets/FiscalNote/billsum/{splits['train']}")
        
        # Convert to Dataset format
        from datasets import Dataset
        dataset = Dataset.from_pandas(df)
        
        # Sample if needed
        if CONFIG['sample_size'] and CONFIG['sample_size'] < len(dataset):
            dataset = dataset.shuffle(seed=CONFIG['seed']).select(range(CONFIG['sample_size']))
            
        logger.info("Dataset loaded successfully using pandas alternative method")
        
    except Exception as e2:
        logger.error(f"Alternative loading method also failed: {str(e2)}")
        raise RuntimeError(f"Both loading methods failed. Original error: {str(e)}, Alternative error: {str(e2)}")
        
if dataset is None:
    logger.critical("FATAL: Dataset could not be loaded by any implemented method.")
    raise RuntimeError("Dataset loading failed. Please check the dataset name and parameters.")


logger.info(f"Dataset loaded. Original columns: {dataset.column_names}")
logger.info(f"Number of samples in loaded dataset: {len(dataset)}")

if 'text' in dataset.column_names:
    dataset = dataset.rename_column('text', 'article')
    logger.info("Renamed dataset column: 'text' -> 'article'")
    CONFIG['text_col'] = 'article'
    logger.info(f"Updated CONFIG['text_col'] to '{CONFIG['text_col']}'")
else:
    logger.warning(f"'text' column not found in dataset columns: {dataset.column_names}. Skipping rename. Current text column in CONFIG: {CONFIG['text_col']}")

logger.info("Creating dataset splits...")
try:
    total_size = len(dataset)
    logger.info(f"Total dataset size for splitting: {total_size}")

    train_size = int(total_size * CONFIG["split_train_frac"])
    eval_size = total_size - train_size

    if train_size <= 0 or eval_size <= 0:
        logger.error(f"Calculated train_size ({train_size}) or eval_size ({eval_size}) is non-positive. Aborting split.")
        raise ValueError("Train or evaluation set size is not positive. Check dataset size and split_train_frac.")

    dataset_shuffled = dataset.shuffle(seed=CONFIG["seed"]) # Shuffle before selecting
    train_dataset = dataset_shuffled.select(range(train_size))
    eval_dataset = dataset_shuffled.select(range(train_size, total_size))

    logger.info(f"Created splits - Train: {len(train_dataset)}, Eval: {len(eval_dataset)}")
except Exception as e:
    logger.error(f"Error creating dataset splits: {e}")
    raise

logger.info("Processing datasets (tokenization, etc.)...")
try:
    tokenized_datasets = {
        'train': train_dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=train_dataset.column_names
        ),
        'eval': eval_dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=eval_dataset.column_names
        )
    }
    logger.info("Dataset processing complete.")
    logger.info(f"Final tokenized dataset sizes - Training samples: {len(tokenized_datasets['train'])}, Evaluation samples: {len(tokenized_datasets['eval'])}")
except Exception as e:
    logger.error(f"Error processing datasets: {e}")
    raise

## 10. Define Training Arguments

Configure the training process using Seq2SeqTrainingArguments.


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=CONFIG["output_dir"],
    num_train_epochs=CONFIG["num_train_epochs"],
    per_device_train_batch_size=CONFIG["per_device_train_batch_size"],
    per_device_eval_batch_size=CONFIG["per_device_eval_batch_size"],
    gradient_accumulation_steps=4,  
    learning_rate=CONFIG["learning_rate"],
    weight_decay=CONFIG["weight_decay"],
    warmup_steps=CONFIG["warmup_steps"],
    
    # Optimization
    fp16=CONFIG["fp16"],
    fp16_full_eval=True,
    bf16=False,
    optim="adafactor",
    
    # Logging & Evaluation
    logging_dir=f"{CONFIG['output_dir']}/logs",
    logging_strategy="steps",
    logging_steps=50,  # More frequent logging
    eval_strategy=CONFIG["evaluation_strategy"],
    eval_steps=100,
    
    # Saving
    save_strategy="steps",
    save_steps=100,  # Match with eval_steps
    save_total_limit=3,  # Keep three checkpoints
    
    # Model Loading
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    
    # Generation
    predict_with_generate=True,
    generation_max_length=CONFIG["max_target_tokens"],
    generation_num_beams=CONFIG["gen_num_beams"],
    
    # Other
    report_to=CONFIG["report_to"],
    seed=CONFIG["seed"],
    gradient_checkpointing=CONFIG["gradient_checkpointing"],
    overwrite_output_dir=CONFIG["overwrite_output_dir"],
)

logger.info(f"Training arguments: {training_args}")


## 11. Define Metrics Computation

Function to compute ROUGE and BLEU scores for evaluation.


In [ ]:
from functools import lru_cache
import numpy as np
import nltk
from typing import Dict, List

@lru_cache(maxsize=1)
def get_metrics():
    """Load and cache evaluation metrics."""
    return {
        "rouge": evaluate.load("rouge"),
        "bleu": evaluate.load("bleu"),
        "bertscore": evaluate.load("bertscore")
    }

def process_texts(texts: List[str]) -> List[str]:
    """Clean and process texts for evaluation."""
    return ["\n".join(nltk.sent_tokenize(text.strip())) for text in texts]

def compute_metrics(eval_preds, batch_size: int = 32) -> Dict[str, float]:
    """Compute evaluation metrics with improved error handling and statistics."""
    try:
        metrics = get_metrics()
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]

        # Decode predictions and labels
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Process texts
        decoded_preds = process_texts(decoded_preds)
        decoded_labels = process_texts(decoded_labels)

        # Compute metrics
        rouge_results = metrics["rouge"].compute(
            predictions=decoded_preds, 
            references=decoded_labels
        )
        
        decoded_labels_bleu = [[label] for label in decoded_labels]
        bleu_results = metrics["bleu"].compute(
            predictions=decoded_preds, 
            references=decoded_labels_bleu
        )

        # Compute additional statistics
        pred_lengths = [len(p.split()) for p in decoded_preds]
        ref_lengths = [len(r.split()) for r in decoded_labels]

        results = {
            "rouge1": rouge_results["rouge1"],
            "rouge2": rouge_results["rouge2"],
            "rougeL": rouge_results["rougeL"],
            "rougeLsum": rouge_results["rougeLsum"],
            "bleu": bleu_results["bleu"],
            "avg_pred_length": np.mean(pred_lengths),
            "avg_ref_length": np.mean(ref_lengths),
            "compression_ratio": np.mean([p/r for p, r in zip(pred_lengths, ref_lengths)])
        }

        # Add generation length metric
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        results["gen_len"] = np.mean(prediction_lens)

        return {k: round(v, 4) for k, v in results.items()}

    except Exception as e:
        logger.error(f"Error computing metrics: {e}")
        return {
            "rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0,
            "rougeLsum": 0.0, "bleu": 0.0, "gen_len": 0,
            "error": str(e)
        }

logger.info("Enhanced metrics computation function defined.")

## 11. Initialize Trainer

Set up the `Seq2SeqTrainer` with the model, arguments, datasets, tokenizer, and metrics function.


In [ ]:
from transformers import EarlyStoppingCallback,TrainerCallback
import gc
import torch


def clear_memory():
    """Clear unused memory before training"""
    gc.collect()
    torch.cuda.empty_cache()
    print_memory_usage()

class MemoryTrackingCallback(TrainerCallback):
    """Callback to track memory usage during training"""
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0:  # Monitor every 100 steps
            print_memory_usage()

def validate_training_args(args, model):
    """Validate training arguments for potential issues"""
    if args.per_device_train_batch_size * args.gradient_accumulation_steps > 64:
        logger.warning("Total batch size might be too large for available memory")
    
    if args.fp16 and not torch.cuda.is_available():
        raise ValueError("FP16 requires CUDA")

# Clear memory before initialization
clear_memory()

# Initialize data collator with error handling
try:
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=tokenizer.pad_token_id,
        pad_to_multiple_of=8 if CONFIG["fp16"] else None
    )
    logger.info("Data collator initialized successfully")
except Exception as e:
    logger.error(f"Failed to initialize data collator: {e}")
    raise

# Validate training arguments
validate_training_args(training_args, model)

# Initialize trainer with enhanced monitoring
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=3),
        MemoryTrackingCallback()
    ]
)

logger.info("Trainer initialized with enhanced monitoring")
print_memory_usage()

## 12. Train the Model

Start the fine-tuning process. This will take some time depending on the dataset size and hardware. 🥳


In [ ]:
def get_latest_checkpoint(checkpoint_dir):
    """Find most recent checkpoint in the directory"""
    checkpoints = [d for d in os.listdir(checkpoint_dir) 
                  if d.startswith("checkpoint-")]
    if not checkpoints:
        return None
    return os.path.join(checkpoint_dir, 
                       sorted(checkpoints, key=lambda x: int(x.split("-")[1]))[-1])

In [ ]:
logger.info("Starting training...")
try:
    # Setup checkpoint directory
    checkpoint_dir = os.path.join(CONFIG["output_dir"], "checkpoints")
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Check for existing checkpoint
    resume_checkpoint = get_latest_checkpoint(checkpoint_dir)
    if resume_checkpoint:
        logger.info(f"Resuming from checkpoint: {resume_checkpoint}")
    
    # Print training info
    logger.info("Training Configuration:")
    logger.info(f"- Number of training examples: {len(trainer.train_dataset)}")
    logger.info(f"- Number of validation examples: {len(trainer.eval_dataset)}")
    logger.info(f"- Training Epochs: {CONFIG['num_train_epochs']}")
    logger.info(f"- Batch size: {CONFIG['per_device_train_batch_size']}")
    
    # Execute training
    train_result = trainer.train(resume_from_checkpoint=resume_checkpoint)
    
    # Log final metrics
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    
    logger.info("Training completed successfully!")
    logger.info(f"Final Training Loss: {metrics.get('train_loss', 'N/A')}")
    
except Exception as e:
    logger.error(f"Training failed: {e}")
    if wandb.run:
        wandb.log({"training_error": str(e)})
        wandb.run.finish(exit_code=1)
    raise e

finally:
    print_memory_usage()
    torch.cuda.empty_cache()

## 13. Evaluate the Model

Evaluate the fine-tuned model on the evaluation set to get final performance metrics.


In [ ]:
logger.log("Evaluating model...")
eval_metrics = trainer.evaluate()

logger.info("Evaluation metrics:")
for key, value in eval_metrics.items():
    logger.info(f"{key}: {value}")

# Log evaluation metrics
trainer.log_metrics("eval", eval_metrics)
trainer.save_metrics("eval", eval_metrics) # Saves to all_results.json

# Prepare the training_report.json
training_report = {
    "model_name": CONFIG["model_name"],
    "dataset_name": CONFIG["dataset_name"],
    "lora_adapter_name": CONFIG["lora_adapter_name"],
    "output_directory": CONFIG["output_dir"],
    "training_arguments": {k: str(v) if isinstance(v, (torch.device, BitsAndBytesConfig)) else v for k, v in training_args.to_dict().items()}, # Convert non-serializable items
    "train_metrics": trainer.state.log_history[:-1], # All logged steps except final eval
    "eval_metrics": eval_metrics,
    "final_training_loss": trainer.state.log_history[-2].get('loss') if len(trainer.state.log_history) > 1 and 'loss' in trainer.state.log_history[-2] else trainer.state.log_history[-1].get('train_loss', 'N/A')
}


# Add ROUGE and BLEU from eval_metrics to the top level for easier access
for metric_key in ["eval_rouge1", "eval_rouge2", "eval_rougeL", "eval_bleu"]:
    if metric_key in eval_metrics:
        training_report[metric_key.replace("eval_", "")] = eval_metrics[metric_key]


# Save training_report.json locally
report_path = os.path.join(CONFIG["output_dir"], CONFIG["training_report_filename"])
with open(report_path, "w") as f:
    json.dump(training_report, f, indent=4)
logger.info(f"Training report saved to {report_path}")

if wandb.run:
    wandb.log(eval_metrics) # Log final eval metrics
    wandb.save(report_path) # Save report to W&B artifacts
    logger.info("Evaluation metrics and report logged to W&B.")

print_memory_usage()

## 14. Save Model and LoRA Adapter

Save the fine-tuned LoRA adapter and the full model if needed.


In [ ]:
# Save the LoRA adapter
lora_adapter_path = os.path.join(CONFIG["output_dir"], CONFIG["lora_adapter_name"])
model.save_pretrained(lora_adapter_path) # Saves only the LoRA adapter
tokenizer.save_pretrained(lora_adapter_path) # Save tokenizer with adapter
logger.info(f"LoRA adapter and tokenizer saved to {lora_adapter_path}")

# To save the full model (optional, requires more space)
# merged_model_path = os.path.join(CONFIG["output_dir"], "merged_model_flan_t5_base_billsum")
# try:
#     # Merge LoRA weights with the base model
#     merged_model = model.merge_and_unload()
#     merged_model.save_pretrained(merged_model_path)
#     tokenizer.save_pretrained(merged_model_path)
#     logger.info(f"Full merged model saved to {merged_model_path}")
# except Exception as e:
#     logger.error(f"Could not merge and save full model: {e}. This might happen if the base model is not fully on CPU or due to memory constraints.")
#     logger.info("Only the LoRA adapter was saved.")


# If Google Drive is mounted, copy outputs there
if CONFIG["mount_drive"] and os.path.exists(CONFIG["gdrive_output_dir"]):
    logger.info(f"Copying outputs to Google Drive: {CONFIG['gdrive_output_dir']}")
    # Copy LoRA adapter
    gdrive_lora_path = os.path.join(CONFIG["gdrive_output_dir"], CONFIG["lora_adapter_name"])
    if os.path.exists(gdrive_lora_path):
        logger.info(f"Removing existing LoRA adapter from GDrive: {gdrive_lora_path}")
        os.system(f"rm -rf '{gdrive_lora_path}'") # Use os.system for `rm -rf`
    os.system(f"cp -r '{lora_adapter_path}' '{CONFIG['gdrive_output_dir']}/'")
    logger.info(f"LoRA adapter copied to {gdrive_lora_path}")

    # Copy training report
    gdrive_report_path = os.path.join(CONFIG["gdrive_output_dir"], CONFIG["training_report_filename"])
    os.system(f"cp '{report_path}' '{gdrive_report_path}'")
    logger.info(f"Training report copied to {gdrive_report_path}")

    # Copy all_results.json (contains eval metrics)
    all_results_path = os.path.join(CONFIG["output_dir"], "all_results.json")
    if os.path.exists(all_results_path):
        gdrive_all_results_path = os.path.join(CONFIG["gdrive_output_dir"], "all_results.json")
        os.system(f"cp '{all_results_path}' '{gdrive_all_results_path}'")
        logger.info(f"all_results.json copied to {gdrive_all_results_path}")

    # If merged model was saved and exists, copy it too
    # if 'merged_model' in locals() and os.path.exists(merged_model_path):
    #     gdrive_merged_model_path = os.path.join(CONFIG["gdrive_output_dir"], "merged_model_flan_t5_base_billsum")
    #     if os.path.exists(gdrive_merged_model_path):
    #         logger.info(f"Removing existing merged model from GDrive: {gdrive_merged_model_path}")
    #         os.system(f"rm -rf '{gdrive_merged_model_path}'")
    #     os.system(f"cp -r '{merged_model_path}' '{CONFIG['gdrive_output_dir']}/'")
    #     logger.info(f"Full merged model copied to {gdrive_merged_model_path}")
else:
    logger.warning("Google Drive not mounted or GDrive output path does not exist. Outputs saved locally.")

if wandb.run:
    # Log LoRA adapter as artifact if desired
    # lora_artifact = wandb.Artifact(CONFIG["lora_adapter_name"], type="model")
    # lora_artifact.add_dir(lora_adapter_path)
    # wandb.log_artifact(lora_artifact)
    # logger.info(f"LoRA adapter logged as W&B artifact: {CONFIG['lora_adapter_name']}")
    wandb.finish()

logger.info("Script finished.")

## Test the model with a sample input


In [ ]:
# Test document - Cybersecurity and Privacy Protection Act
test_document = """
CYBERSECURITY AND PRIVACY PROTECTION ACT OF 2025

SECTION 1. SHORT TITLE AND PURPOSE

    (a) This Act may be cited as the 'Cybersecurity and Privacy Protection Act of 2025'.
    (b) The purpose of this Act is to enhance cybersecurity measures and protect individual privacy in the digital age.

SECTION 2. DEFINITIONS

In this Act:
    (1) 'Personal Data' means any information relating to an identified or identifiable natural person.
    (2) 'Data Controller' means any entity that determines the purposes and means of processing personal data.
    (3) 'Critical Infrastructure' means systems and assets vital to national security.

SECTION 3. CYBERSECURITY REQUIREMENTS

    (a) MANDATORY SECURITY MEASURES.—
        (1) All Data Controllers shall implement:
            (A) End-to-end encryption for data transmission
            (B) Multi-factor authentication for system access
            (C) Regular security audits and vulnerability assessments

    (b) INCIDENT REPORTING.—
        (1) Data Controllers shall report any security breach within 48 hours.
        (2) Penalties for non-compliance shall be up to $500,000 per incident.

SECTION 4. PRIVACY PROTECTIONS

    (a) CONSENT REQUIREMENTS.—
        (1) Explicit consent required for data collection
        (2) Right to access and delete personal data
        (3) Annual privacy impact assessments

    (b) CHILDREN'S PRIVACY.—
        (1) Enhanced protections for users under 13
        (2) Parental consent requirements

SECTION 5. ENFORCEMENT

    (a) The Federal Trade Commission shall enforce this Act.
    (b) State Attorneys General may bring civil actions.

SECTION 6. AUTHORIZATION OF APPROPRIATIONS

    There is authorized to be appropriated $275,000,000 for fiscal year 2026 to carry out this Act.
"""

# Test the preprocessing function

## adding trained lora adapter to the model
model = AutoModelForSeq2SeqLM.from_pretrained(
    lora_adapter_path,
    device_map="auto",
    torch_dtype=torch.float16
)
model = get_peft_model(model, lora_config)
model.load_adapter(lora_adapter_path, CONFIG["lora_adapter_name"])
model.set_adapter(CONFIG["lora_adapter_name"])
model.eval()
model.to("cuda:0")
# Tokenize the test document
inputs = tokenizer(test_document, return_tensors="pt", padding=True, truncation=True).to("cuda:0")
# Generate summary
with torch.no_grad():
    outputs = model.generate(**inputs)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Summary:")
print(summary)
print_memory_usage()
